# Cartoon Filter using OpenCV

Author: Mohamed Oussama NAJI

Date: March 29, 2024

## Table of Contents
1. [Introduction](#introduction)
2. [Loading Haar Cascade for Face Detection](#loading-haar-cascade)
3. [Loading and Preprocessing the Mask Image](#loading-mask-image)
4. [Video Capture and Processing](#video-capture-processing)
5. [Saving the Output Video](#saving-output-video)
6. [Results](#results)
7. [Conclusion](#conclusion)

## Introduction <a id="introduction"></a>

In this notebook, we will create a cartoon filter using OpenCV. The filter will detect faces in a video stream and overlay a Spider-Man mask on the detected faces in real-time. We will use the Haar cascade classifier for face detection and image processing techniques to blend the mask with the video frames.


## Loading Haar Cascade for Face Detection <a id="loading-haar-cascade"></a>

We're using a Haar cascade classifier for detecting faces in the video. The `haarcascade_frontalface_alt2.xml` is a pre-trained face detector provided by OpenCV.


In [ ]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt

path_face_cascade = os.path.join(cv2.__file__.rsplit('python', 1)[0], 'data', 'haarcascade_frontalface_alt2.xml')
face_cascade = cv2.CascadeClassifier(path_face_cascade)


## Loading and Preprocessing the Mask Image <a id="loading-mask-image"></a>

In [ ]:
# Load the Spider-Man mask image
mask_image = cv2.imread('spiderman2.png', cv2.IMREAD_UNCHANGED)

# If the image has 3 channels (B, G, R)
if mask_image.shape[2] == 3:
    # Add an alpha channel
    mask_image = cv2.cvtColor(mask_image, cv2.COLOR_BGR2BGRA)

# Set all white (also shades of whites) pixels to be transparent
white_area = cv2.inRange(mask_image, (200, 200, 200, 255), (255, 255, 255, 255))
mask_image[white_area > 0] = (255, 255, 255, 0)

# Save the image
cv2.imwrite('hanchoufou.png', mask_image)

# Load the final image
final_image = cv2.imread('hanchoufou.png', cv2.IMREAD_UNCHANGED)

# Convert the image from BGR to RGBA
final_image = cv2.cvtColor(final_image, cv2.COLOR_BGRA2RGBA)
mask_image = final_image

# Display the image
plt.imshow(final_image)
plt.show()

print(mask_image.shape)

# If the image has 3 channels (B, G, R)
if mask_image.shape[2] == 3:
    # Add an alpha channel
    mask_image = cv2.cvtColor(mask_image, cv2.COLOR_BGR2BGRA)

unique_values = np.unique(mask_image[:, :, 3])
print(unique_values)

# Resize the image
scale_percent = 200  # percent of original size
width = int(mask_image.shape[1] * scale_percent / 100)
height = int(mask_image.shape[0] * scale_percent / 100)
dim = (width, height)
resized = cv2.resize(mask_image, dim, interpolation = cv2.INTER_AREA)

# Display the image
plt.imshow(cv2.cvtColor(resized, cv2.COLOR_BGRA2RGBA))
plt.show()


## Video Capture and Processing <a id="video-capture-processing"></a>

Here we start capturing video from the default camera.


In [ ]:
video_capture = cv2.VideoCapture(0)

# Define the codec and create VideoWriter object to save the video
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.avi', fourcc, 20.0, (640, 480))

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

while True:
    # Capture frame-by-frame
    ret, frame = video_capture.read()
    if not ret:
        break  # If no frame is captured, break the loop

    # Convert to grayscale
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    detected_faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in detected_faces:
        # Resize mask to fit the face
        mask_resized = cv2.resize(mask_image, (w, h))

        # Take the alpha layer from the resized mask and create the inverse mask
        mask_alpha = mask_resized[:, :, 3]
        mask_inv = cv2.bitwise_not(mask_alpha)

        # Take the color channels of the mask
        mask_color = mask_resized[:, :, :3]

        # Extract the region of interest (ROI) from the frame
        roi = frame[y:y+h, x:x+w]

        # Use the masks to blend the mask and the background
        face_background = cv2.bitwise_and(roi, roi, mask=mask_inv)
        mask_foreground = cv2.bitwise_and(mask_color, mask_color, mask=mask_alpha)

        # Combine the background and the foreground
        dst = cv2.add(face_background, mask_foreground)

        # Place the combined image back onto the frame
        frame[y:y+h, x:x+w] = dst

    # Write the frame into the file 'output.avi'
    out.write(frame)

    # Display the resulting frame
    cv2.imshow('Video with Spider-Man Mask', frame)

    # Save a screenshot if 's' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('s'):
        cv2.imwrite('/path/to/save/screenshot.png', frame)

    # Exit loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


## Saving the Output Video <a id="saving-output-video"></a>

In [ ]:
# Release everything when job is finished
video_capture.release()
out.release()
cv2.destroyAllWindows()

## Results <a id="results"></a>

The cartoon filter was successfully implemented using OpenCV. The program captures video from the default camera and detects faces in real-time using the Haar cascade classifier. For each detected face, the Spider-Man mask is resized to fit the face and blended with the video frame using image processing techniques.

The resulting video with the cartoon filter applied is displayed in a window titled 'Video with Spider-Man Mask'. The user can save a screenshot of the current frame by pressing the 's' key and exit the program by pressing the 'q' key.

The output video is saved as 'output.avi' in the same directory as the notebook.

## Conclusion <a id="conclusion"></a>

In this notebook, we developed a cartoon filter using OpenCV to overlay a Spider-Man mask on detected faces in a video stream. We utilized the Haar cascade classifier for face detection and applied image processing techniques to blend the mask with the video frames.

The cartoon filter provides an entertaining and interactive way to modify video streams in real-time. It can be further enhanced by adding more mask options, improving the face detection accuracy, and optimizing the performance for real-time processing.

This project demonstrates the power of OpenCV for image and video processing tasks and showcases the potential for creating fun and engaging applications using computer vision techniques.